Get Live Jobs From Google Search and save them as raw data to jobs.csv for further cleaning and processing. DONT RUN AGAIN

In [ ]:
import csv
import os
import requests
import pandas as pd

# SerpAPI endpoint
API_URL = "https://serpapi.com/search"
SERPAPI_KEY = "0b2ab1b29442c1b8003176083e190527f9695bc0de45fc16a0afad683298bb02"

PARAMS = {
    "engine": "google_jobs",
    "q": "Data Science Intern",
    "hl": "en",
    "gl": "us",
    "api_key": SERPAPI_KEY,
    "tbs": "qdr:d"
}

def fetch_google_jobs_to_csv():
    response = requests.get(API_URL, params=PARAMS)
    
    if response.status_code != 200:
        print("Error:", response.status_code)
        return

    jobs = response.json().get("jobs_results", [])
    if not jobs:
        print("No jobs found.")
        return

    job_list = []
    for job in jobs:
        job_title = job.get("title", "N/A")
        company = job.get("company_name", "N/A")
        location = job.get("location", "N/A")
        job_description = job.get("description", "No description available").replace('\n', ' ').strip()
        job_id = job.get("job_id", "")
        job_url = f"https://www.google.com/search?q={job_id}" if job_id else "N/A"

        job_list.append({
            "Job_Title": job_title,
            "Company": company,
            "Location": location,
            "Job_Description": job_description,
            "Job_URL": job_url
        })

    # Convert to DataFrame
    base_path = os.path.join(".", "Jobs")
    df = pd.DataFrame(job_list)
    
    # Save to CSV
    _path = os.path.join(base_path, "jobs.csv")
    df.to_csv(_path, index=False, encoding='utf-8', quoting=csv.QUOTE_ALL)
    print(f"Saved {len(df)} job listings to 'jobs.csv'.")
    

# Run the function
fetch_google_jobs_to_csv()


Check Skills required in each job

In [4]:
import pandas as pd
import re

# Load job descriptions
df = pd.read_csv("jobs.csv")
df["Job_Description"] = df["Job_Description"].astype(str)

# Initialize lists to store extracted components
all_skills = []
all_responsibilities = []
all_qualifications = []

# Regular expression based parsing for each job description
for desc in df["Job_Description"]:
    # Extract responsibilities
    responsibilities_match = re.search(r"RESPONSIBILITIES:(.*?)(?:MINIMUM QUALIFICATIONS:|QUALIFICATIONS:|SKILLS:|LOCATION:|PAY:|APPLICATION DEADLINE:|JOB ID:|$)", desc, re.IGNORECASE | re.DOTALL)
    responsibilities = responsibilities_match.group(1).strip() if responsibilities_match else ""

    # Extract qualifications
    qualifications_match = re.search(r"(?:MINIMUM QUALIFICATIONS:|QUALIFICATIONS:)(.*?)(?:SKILLS:|RESPONSIBILITIES:|LOCATION:|PAY:|APPLICATION DEADLINE:|JOB ID:|$)", desc, re.IGNORECASE | re.DOTALL)
    qualifications = qualifications_match.group(1).strip() if qualifications_match else ""

    # Extract skills
    skills_match = re.search(r"SKILLS\s*:\s*(.*?\.)(?=QUALIFICATIONS:|RESPONSIBILITIES:|LOCATION:|PAY:|APPLICATION DEADLINE:|JOB ID:|$)", desc, re.IGNORECASE | re.DOTALL)
    skills = skills_match.group(1).strip() if skills_match else ""


    # Append results to lists
    all_skills.append(skills)
    all_responsibilities.append(responsibilities)
    all_qualifications.append(qualifications)

# Create extracted DataFrame
extracted_data = pd.DataFrame({
    "Skills": all_skills,
    "Responsibilities": all_responsibilities,
    "Qualifications": all_qualifications
})

# ----------- PRINT OUTPUT ----------
print("======= Extracted Job Info =======")
for i in range(len(extracted_data)):
    print(f"Job {i+1}")
    print("Skills:")
    print(extracted_data.loc[i, 'Skills'] or "  - Not specified")

    print("\nResponsibilities:")
    print(extracted_data.loc[i, 'Responsibilities'] or "  - Not specified")

    print("\nQualifications:")
    print(extracted_data.loc[i, 'Qualifications'] or "  - Not specified")

    print("\n" + "-" * 80)


======= Extracted Job Info =======
Job 1
Skills:
LLMs via APIs,Understanding of application of dense vector,text embedding models for finding semantic similarity,NoSQL,Vector Database,Graph construction,AWS,Azure,Python,Data visualization,matplotlib,plotly,Power BI,Infrastructure as code.

Responsibilities:
Collaborate with cross-functional teams to identify business challenges and opportunities where data science can provide valuable insights,Assist in developing and implementing advanced analytical models and methodologies to analyze complex data sets Support data visualization efforts to effectively communicate insights to stakeholders,Conduct data analysis and provide actionable recommendations based on data-driven insights,Stay updated with the latest trends and advancements in data science and identify opportunities to leverage new tools and technologies.

Qualifications:
Currently pursuing a degree in Data Science, Computer Science, Statistics, or a related fieldStrong analytica

In [7]:
import pandas as pd
import re
from collections import Counter
import plotly.express as px

# Load job descriptions
df = pd.read_csv("jobs.csv")
df["Job_Description"] = df["Job_Description"].astype(str)

# Extract skills
all_skills = []
for desc in df["Job_Description"]:
    skills_match = re.search(
        r"SKILLS\s*:\s*(.*?)(?:\.|QUALIFICATIONS:|RESPONSIBILITIES:|LOCATION:|PAY:|APPLICATION DEADLINE:|JOB ID:|$)",
        desc, re.IGNORECASE | re.DOTALL
    )
    skills = skills_match.group(1).strip() if skills_match else ""
    all_skills.append(skills)

# Add extracted skills to dataframe
df["Extracted_Skills"] = all_skills

# Prepare for plotting
skill_records = []
for idx, row in df.iterrows():
    job_id = f"Job {idx+1}"
    skills = [s.strip().lower() for s in re.split(r",|;", row["Extracted_Skills"]) if s.strip()]
    freq = Counter(skills)
    for skill, count in freq.items():
        skill_records.append({"Job": job_id, "Skill": skill, "Count": count})

skill_df = pd.DataFrame(skill_records)

# Plot using Plotly
fig = px.bar(
    skill_df,
    x="Skill",
    y="Count",
    color="Job",
    barmode="group",
    title="Skill Frequency per Job (Extracted from CSV)",
    hover_data=["Job", "Count"],
    labels={"Skill": "Skill", "Count": "Frequency"}
)
fig.update_layout(xaxis_tickangle=45, height=600)
fig.show()


In [8]:
import pandas as pd
import re
from collections import Counter
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Load the job descriptions
df = pd.read_csv("jobs.csv")
df["Job_Description"] = df["Job_Description"].astype(str)

# Extract skills section from each job description
all_skills = []
for desc in df["Job_Description"]:
    skills_match = re.search(
        r"SKILLS\s*:\s*(.*?)(?:\.|QUALIFICATIONS:|RESPONSIBILITIES:|LOCATION:|PAY:|APPLICATION DEADLINE:|JOB ID:|$)",
        desc, re.IGNORECASE | re.DOTALL
    )
    skills = skills_match.group(1).strip() if skills_match else ""
    all_skills.append(skills)

# Add extracted skills column
df["Extracted_Skills"] = all_skills

# Create pie charts
job_count = len(df)
cols = 2
rows = (job_count + 1) // cols

fig = make_subplots(
    rows=rows, cols=cols, specs=[[{'type': 'domain'}]*cols]*rows,
    subplot_titles=[f"Job {i+1}" for i in range(job_count)]
)

for idx, skills in enumerate(df["Extracted_Skills"]):
    job_id = f"Job {idx+1}"
    skill_list = [s.strip().lower() for s in re.split(r",|;", skills) if s.strip()]
    if not skill_list:
        continue
    skill_freq = Counter(skill_list)
    row = (idx // cols) + 1
    col = (idx % cols) + 1
    fig.add_trace(
        go.Pie(labels=list(skill_freq.keys()), values=list(skill_freq.values()), name=job_id),
        row=row, col=col
    )

fig.update_layout(title_text="Skill Distribution per Job (Pie Charts)", height=300 * rows)
fig.show()


In [11]:
import pandas as pd
import re
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Load job descriptions
df = pd.read_csv("jobs.csv")
df["Job_Description"] = df["Job_Description"].astype(str)

# Extract qualifications
qualifications_list = []
for desc in df["Job_Description"]:
    match = re.search(r"(?:MINIMUM QUALIFICATIONS:|QUALIFICATIONS:)(.*?)(?:SKILLS:|RESPONSIBILITIES:|LOCATION:|PAY:|APPLICATION DEADLINE:|JOB ID:|$)",
                      desc, re.IGNORECASE | re.DOTALL)
    qualifications = match.group(1).strip().lower() if match else "none"
    qualifications_list.append(qualifications)

# Create a DataFrame for qualifications
qual_df = pd.DataFrame({
    "Job": [f"Job {i+1}" for i in range(len(df))],
    "Qualifications": qualifications_list
})

# Break down the qualification text into separate keywords
qual_keywords = []
for i, row in qual_df.iterrows():
    items = re.split(r"[,\.\n]", row["Qualifications"])
    for item in items:
        clean_item = item.strip()
        if clean_item:
            qual_keywords.append({"Job": row["Job"], "Qualification": clean_item})

qual_keyword_df = pd.DataFrame(qual_keywords)

# Pie chart setup
jobs = qual_keyword_df["Job"].unique()
rows = (len(jobs) + 1) // 2
fig = make_subplots(rows=rows, cols=2, specs=[[{'type': 'domain'}]*2]*rows,
                    subplot_titles=jobs)

# Add pie charts
for idx, job in enumerate(jobs):
    row = (idx // 2) + 1
    col = (idx % 2) + 1
    job_data = qual_keyword_df[qual_keyword_df["Job"] == job]
    count_data = job_data["Qualification"].value_counts()
    fig.add_trace(go.Pie(labels=count_data.index, values=count_data.values, name=job),
                  row=row, col=col)

# Layout
fig.update_layout(title_text="Qualification Distribution per Job", height=300 * rows)
fig.show()
